# WAVE Test Experiment Cleanup

This notebook performs blanket deletion of ALL experiments with the "test" tag.

**⚠️ ADMIN ONLY**: This notebook requires an ADMIN_API_KEY with deletion permissions.

## What this does:
1. Finds all experiments tagged with "test"
2. Shows data counts for each experiment  
3. **DELETES ALL test experiments and their data**
4. Optionally deletes test experiment types

**Prerequisites**: Add ADMIN_API_KEY to your tools/.env file

In [1]:
from utils import load_admin_environment_variables, get_user_confirmation
from wave_client import WaveClient
import pandas as pd
import sys

In [2]:
# Load admin environment variables
print("⚠️  ADMIN CLEANUP NOTEBOOK")
print("This requires ADMIN_API_KEY with deletion permissions\n")

ADMIN_API_KEY, WAVE_BACKEND_URL = load_admin_environment_variables()

print(f"📡 Backend URL: {WAVE_BACKEND_URL}")
print(f"🔑 Using ADMIN key ending in: ...{ADMIN_API_KEY[-4:]}")
print("✅ Admin credentials loaded")

⚠️  ADMIN CLEANUP NOTEBOOK
This requires ADMIN_API_KEY with deletion permissions

Present working directory: /Users/doug/Documents/code/wave/experiment-template/tools
Loading environment variables from /Users/doug/Documents/code/wave/experiment-template/tools/.env
📡 Backend URL: https://wave-backend-production-8781.up.railway.app
🔑 Using ADMIN key ending in: ...Mm45
✅ Admin credentials loaded


In [3]:
# Find all test experiments (blanket deletion approach)
print("🔍 Finding ALL experiments with 'test' tag for deletion...\n")


async def get_test_experiments():
    async with WaveClient(api_key=ADMIN_API_KEY, base_url=WAVE_BACKEND_URL) as client:
        try:
            all_experiments = await client.experiments.list(skip=0, limit=1000)
            # Blanket selection: ALL experiments with 'test' tag will be deleted
            test_experiments = [exp for exp in all_experiments if "test" in exp.get("tags", [])]
            return test_experiments
        except Exception as e:
            print(f"❌ Error fetching experiments: {e}")
            sys.exit("Failed to connect - check your ADMIN_API_KEY")


test_experiments = await get_test_experiments()

if not test_experiments:
    print("✅ No test experiments found - nothing to clean up!")
else:
    print(f"⚠️  Found {len(test_experiments)} experiments tagged 'test' - ALL WILL BE DELETED:")

    for i, exp in enumerate(test_experiments, 1):
        created_date = exp.get("created_at", "Unknown")[:10]
        print(f"  {i}. {exp['description']}")
        print(f"     ID: {exp['uuid']} | Created: {created_date}")
        print(f"     Tags: {', '.join(exp.get('tags', []))}")
        print()

🔍 Finding ALL experiments with 'test' tag for deletion...

✅ No test experiments found - nothing to clean up!


In [4]:
# Blanket deletion of all test experiments
if test_experiments:
    print("🔍 Checking data counts for deletion...\n")

    # Get data counts for each test experiment
    experiment_data = []

    async with WaveClient(api_key=ADMIN_API_KEY, base_url=WAVE_BACKEND_URL) as client:
        for exp in test_experiments:
            try:
                data_df = await client.experiment_data.get_all_data(experiment_id=exp["uuid"])
                data_count = len(data_df)
                participant_count = data_df["participant_id"].nunique() if data_count > 0 else 0

                experiment_data.append(
                    {
                        "experiment": exp,
                        "data_df": data_df,
                        "data_count": data_count,
                        "participant_count": participant_count,
                    }
                )

                print(f"📋 {exp['description']}")
                print(f"   Data points: {data_count} | Participants: {participant_count}")
                print()

            except Exception as e:
                print(f"⚠️  Could not get data for {exp['uuid']}: {e}")
                experiment_data.append(
                    {
                        "experiment": exp,
                        "data_df": pd.DataFrame(),
                        "data_count": 0,
                        "participant_count": 0,
                    }
                )

    total_data_points = sum(item["data_count"] for item in experiment_data)

    print(f"🚨 BLANKET DELETION WARNING")
    print(f"About to PERMANENTLY DELETE:")
    print(f"  • {len(test_experiments)} test experiments")
    print(f"  • {total_data_points} data points")
    print(f"\n💥 This will delete ALL experiments with 'test' tag - NO EXCEPTIONS")
    print(f"💡 This action cannot be undone!")

In [5]:
if test_experiments and get_user_confirmation(
    "\nProceed with BLANKET deletion of all test experiments?", None
):
    print("\n🚀 Starting blanket deletion...\n")

    deleted_count = 0
    data_deleted_count = 0
    failed_deletions = []

    async with WaveClient(api_key=ADMIN_API_KEY, base_url=WAVE_BACKEND_URL) as client:
        for item in experiment_data:
            exp = item["experiment"]
            data_df = item["data_df"]
            experiment_id = exp["uuid"]

            try:
                print(f"🗑️  Deleting: {exp['description']}")

                # Delete all data first
                if len(data_df) > 0:
                    for _, row in data_df.iterrows():
                        try:
                            await client.experiment_data.delete_row(experiment_id, int(row["id"]))
                            data_deleted_count += 1
                        except Exception as e:
                            print(f"    ⚠️  Failed to delete data row: {e}")

                # Delete experiment
                await client.experiments.delete(experiment_id)
                deleted_count += 1
                print(f"    ✅ Deleted experiment {experiment_id}")

            except Exception as e:
                print(f"    ❌ Failed to delete {experiment_id}: {e}")
                failed_deletions.append(exp)

    print(f"\n📊 Blanket Deletion Summary:")
    print(f"✅ Successfully deleted: {deleted_count}/{len(test_experiments)} experiments")
    print(f"🗑️  Deleted data points: {data_deleted_count}")

    if failed_deletions:
        print(f"❌ Failed to delete {len(failed_deletions)} experiments:")
        for exp in failed_deletions:
            print(f"  - {exp['description']} (ID: {exp['uuid']})")

    print(f"\n✅ Blanket deletion of test experiments complete!")

else:
    print("❌ Blanket deletion cancelled")

❌ Blanket deletion cancelled


In [6]:
# Optional: Clean up test experiment types
print("\n🔍 Checking for test experiment types...")


async def get_test_experiment_types():
    async with WaveClient(api_key=ADMIN_API_KEY, base_url=WAVE_BACKEND_URL) as client:
        try:
            all_types = await client.experiment_types.list(skip=0, limit=1000)
            # Look for types with 'test' in name or starting with 'TEST_'
            test_types = [
                t for t in all_types if "test" in t["name"].lower() or t["name"].startswith("TEST_")
            ]
            return test_types
        except Exception as e:
            print(f"❌ Error fetching experiment types: {e}")
            return []


test_types = await get_test_experiment_types()

if test_types:
    print(f"📋 Found {len(test_types)} test experiment types:")
    for t in test_types:
        print(f"  • {t['name']} (Table: {t['table_name']}) [ID: {t['id']}]")

    if get_user_confirmation("\nDelete these test experiment types too?", None):
        print("\n🗑️  Deleting test experiment types...")

        type_deleted_count = 0
        async with WaveClient(api_key=ADMIN_API_KEY, base_url=WAVE_BACKEND_URL) as client:
            for exp_type in test_types:
                try:
                    await client.experiment_types.delete(exp_type["id"])
                    type_deleted_count += 1
                    print(f"  ✅ Deleted: {exp_type['name']}")
                except Exception as e:
                    print(f"  ❌ Failed to delete {exp_type['name']}: {e}")

        print(f"\n📊 Deleted {type_deleted_count}/{len(test_types)} experiment types")
    else:
        print("ℹ️  Keeping experiment types")
else:
    print("✅ No test experiment types found")

print("\n🎉 All cleanup operations complete!")


🔍 Checking for test experiment types...
✅ No test experiment types found

🎉 All cleanup operations complete!
